In [57]:
import sys
# MC: need to specify path to remo in notebook
# mac version
local_path_to_repo =  '/Users/melodi/remo-python'
# windows version
#local_path_to_repo =  'C:/Users/crows/Documents/GitHub/remo-python'

sys.path.insert(0, local_path_to_repo)

In [58]:
%load_ext autoreload
%autoreload 2
import remo


    (\(\ 
    (>':') Remo server is running: {'version': '0.3.4'}
                


In [59]:
remo.list_datasets() 

[Dataset 1 - 'Vova's dataset',
 Dataset 2 - 'open images detection',
 Dataset 3 - 'Subset of OID',
 Dataset 4 - 'train',
 Dataset 6 - 'test',
 Dataset 12 - 'oid 100',
 Dataset 13 - 'open images sample data',
 Dataset 19 - 'oid building ',
 Dataset 21 - 'oid building and car',
 Dataset 22 - 'oid building car detection',
 Dataset 25 - 'open images data',
 Dataset 26 - 'oid car',
 Dataset 28 - 'oid car and person',
 Dataset 29 - 'test upload']

In [65]:
my_dataset = remo.get_dataset(28)

In [66]:
my_dataset.list_annotation_sets()

[Annotation set 19 - 'Object Detection Actual', task: Object detection, #classes: 5,
 Annotation set 24 - 'Classification Predictions', task: Image classification, #classes: 2]

In [78]:
val = my_dataset[0:len(my_dataset) // 3]

In [79]:
train = my_dataset[len(my_dataset) // 3:]

In [80]:
val.export_annotation_to_csv('val.csv')

In [81]:
train.export_annotation_to_csv('train.csv')

In [73]:
import pandas as pd
df = pd.read_csv('train.csv')

In [74]:
df.head()

,file_name,class,xmin,ymin,xmax,ymax,height,width
0,039b687dbee6ec4a.jpg,/m/01g317,103.863296,283.794432,226.743296,588.800256,768,1024
1,039b687dbee6ec4a.jpg,/m/01g317,383.268864,212.845824,558.811136,626.833920,768,1024
2,039b687dbee6ec4a.jpg,/m/01g317,542.720000,273.553920,664.868864,629.760000,768,1024
3,039b687dbee6ec4a.jpg,/m/01g317,683.154432,242.102784,811.154432,632.686080,768,1024
4,039b687dbee6ec4a.jpg,/m/01g317,733.623296,0.000000,1023.268864,767.268864,768,1024


We need to convert annotation format into .xml for darknet.

In [14]:
def convert_row(row, class_name='car', root_path='/Users/melodi/Docs/images/'):
    class_codes = {'/m/01g317':'Person', '/m/0k4j':'Car'}
    vals = ('train', row['file_name'], root_path + row['file_name'], str(row['width']), str(row['height']),)
    for i in range(len(row['class'])):
        vals = vals + (class_codes[row['class'][i]].lower(),str(row['xmin'][i]), str(row['ymin'][i]), str(row['xmax'][i]), str(row['ymax'][i]),)
    
    result = """<annotation>
    <folder>%s</folder>
    <filename>%s</filename>
    <path>%s</path>
    <source>
        <database>Unknown</database>
    </source>
    <size>
        <width>%s</width>
        <height>%s</height>
        <depth>3</depth>
    </size>
    <segmented>0</segmented>
    """ + """<object>
        <name>%s</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <difficult>0</difficult>
        <bndbox>
            <xmin>%s</xmin>
            <ymin>%s</ymin>
            <xmax>%s</xmax>
            <ymax>%s</ymax>
        </bndbox>
    </object>""" * len(row['class']) +"""
</annotation>
""" 
    result = result % vals
    filename = row['file_name'].split('.')[0]
    path = '/Users/melodi/Docs/annots/' + filename
    f =  open(path + ".xml", "w")
    f.write(result)
    f.close()


We will group by file_name and aggreagate in a list in the case of multiple objects.

In [75]:
df_grouped = df.groupby('file_name')['class'].apply(list).reset_index(name='class')
for column in df.columns[2:]:
    df_new = df.groupby('file_name')[column].apply(list).reset_index(name=column)
    df_grouped = df_grouped.merge(df_new, on=['file_name'])

In [76]:
# height and width is the same
df_grouped['height'] = df_grouped['height'].apply(lambda x: x[0])
df_grouped['width'] = df_grouped['width'].apply(lambda x: x[0])

In [77]:
df.head()

,file_name,class,xmin,ymin,xmax,ymax,height,width
0,039b687dbee6ec4a.jpg,/m/01g317,103.863296,283.794432,226.743296,588.800256,768,1024
1,039b687dbee6ec4a.jpg,/m/01g317,383.268864,212.845824,558.811136,626.833920,768,1024
2,039b687dbee6ec4a.jpg,/m/01g317,542.720000,273.553920,664.868864,629.760000,768,1024
3,039b687dbee6ec4a.jpg,/m/01g317,683.154432,242.102784,811.154432,632.686080,768,1024
4,039b687dbee6ec4a.jpg,/m/01g317,733.623296,0.000000,1023.268864,767.268864,768,1024


In [17]:
df_grouped.apply(convert_row,axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
129    None
130    None
131    None
132    None
133    None
Length: 134, dtype: object

### Darknet

In [32]:
import cv2
from darkflow.net.build import TFNet
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'

In [19]:
options = {"model": "/Users/melodi/remo-python/cfg/yolov2.cfg", 
           "load": "/Users/melodi/remo-python/bin/yolov2.weights",
           "batch": 40,
           "epoch": 1,
           "train": True,
           "verbalise": False,
           "annotation": "/Users/melodi/Docs/annots/",
           "dataset": "/Users/melodi/Docs/images/",
           "labels": "/Users/melodi/remo-python/labels.txt",
          "backup": "/Users/melodi/remo-python/ckpt/"}

In [20]:
tfnet = TFNet(options)

Parsing /Users/melodi/Docs/remo-python/cfg/yolov2.cfg
Loading /Users/melodi/Docs/remo-python/bin/yolov2.weights ...
Successfully identified 203934260 bytes
Finished in 1.413050889968872s
/Users/melodi/Docs/remo-python/cfg/yolov2.cfg loss hyper-parameters:
	H       = 19
	W       = 19
	box     = 5
	classes = 80
	scales  = [1.0, 5.0, 1.0, 1.0]
Instructions for updating:
Use `tf.cast` instead.
Building /Users/melodi/Docs/remo-python/cfg/yolov2.cfg loss

INFO:tensorflow:Summary name /Users/melodi/Docs/remo-python/cfg/yolov2.cfg loss is illegal; using Users/melodi/Docs/remo-python/cfg/yolov2.cfg_loss instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
tfnet.train()


/Users/melodi/Docs/remo-python/cfg/yolov2.cfg parsing /Users/melodi/Docs/annots/
Parsing for ['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'] 
[====================>]100%  09e3c15d73

In [24]:
options = {
 "model": "/Users/melodi/remo-python/cfg/yolov2.cfg", 
   "load": 3,
 'threshold': 0.5,
    "verbalise": False,
    "labels": "/Users/melodi/remo-python/labels.txt",
  "backup": "/Users/melodi/remo-python/ckpt/"}

tfnet2 = TFNet(options)

Parsing /Users/melodi/Docs/remo-python/cfg/yolov2.cfg
Loading None ...
Finished in 0.00019788742065429688s
INFO:tensorflow:Restoring parameters from /Users/melodi/Docs/remo-python/ckpt/yolov2-3


In [25]:
tfnet2.load_from_ckpt()

INFO:tensorflow:Restoring parameters from /Users/melodi/Docs/remo-python/ckpt/yolov2-3


### Visualize Predictions in Remo

We feed our validation set into the model and prepare the results to upload Remo.

In [34]:
import os 
cars = os.listdir('/Users/melodi/remo-python/val/Car')
results = []
for img_file in cars:
    original_img = cv2.imread('/Users/melodi/remo-python/val/Car/'+img_file)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    outputs = tfnet2.return_predict(original_img)
    result = {'file_name':img_file, 'objects':outputs}
    results.append(result)

In [35]:
people = os.listdir('/Users/melodi/remo-python/val/Person')
for img_file in people:
    original_img = cv2.imread('/Users/melodi/remo-python/val/Person/'+img_file)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    outputs = tfnet2.return_predict(original_img)
    result = {'file_name':img_file, 'objects':outputs}
    results.append(result)

Each output consists to a dictionary containing file_name and objects. 

In [71]:
results[0]

{'file_name': '0004d295cfb46842.jpg',
 'objects': [{'label': 'person',
   'confidence': 0.7596794,
   'topleft': {'x': 858, 'y': 411},
   'bottomright': {'x': 927, 'y': 607}},
  {'label': 'car',
   'confidence': 0.53090864,
   'topleft': {'x': 166, 'y': 409},
   'bottomright': {'x': 249, 'y': 451}},
  {'label': 'car',
   'confidence': 0.78686273,
   'topleft': {'x': 547, 'y': 423},
   'bottomright': {'x': 668, 'y': 484}},
  {'label': 'car',
   'confidence': 0.6657906,
   'topleft': {'x': 747, 'y': 424},
   'bottomright': {'x': 901, 'y': 499}}]}

Then we will convert this into dataframe in order to feed into Remo as below.

In [44]:
import pandas as pd
d = {'person':'/m/01g317', 'car':'/m/0k4j'}
df = pd.DataFrame(columns=['file_name','class','xmin','ymin','xmax','ymax'])
for res in results:
    for i in range(len(res['objects'])):
        df.loc[len(df)+1] = [res['file_name'],d.get(res['objects'][i]['label']),res['objects'][i]['topleft']['x'],res['objects'][i]['topleft']['y'], res['objects'][i]['bottomright']['x'], res['objects'][i]['bottomright']['y']]

In [51]:
df.head()

,file_name,class,xmin,ymin,xmax,ymax
1,0004d295cfb46842.jpg,/m/01g317,858,411,927,607
2,0004d295cfb46842.jpg,/m/0k4j,166,409,249,451
3,0004d295cfb46842.jpg,/m/0k4j,547,423,668,484
4,0004d295cfb46842.jpg,/m/0k4j,747,424,901,499
5,006cc2c07c881fe0.jpg,/m/01g317,711,221,752,326


In [53]:
# drop the other classes different than car and person.
df = df.dropna(subset=['class'])

We will upload this .csv file into Remo dataset.

In [55]:
df.to_csv('/Users/melodi/Docs/remo-python/example/detection_preds_encoded.csv',index=False)